In [ ]:
# script: sec_form_d_lookup
# author: Jennifer Lammers Zimmer, jamm@umich.edu
# last updated: 01/27/2023
# This script uses the Edgar Python package to query the SEC database to find
# CIKs for companies in a CSV file. A lookup on names is performed first to find
# the official Edgar name and then Edgar is queried again to find the CIKs with the 
# official names. The final output is parsed with Beautiful Soup and is then
# is written to a CSV file and includes official company name and CIK.

#Data fields requested from the Soup are:
      #filed on date
      #signed_date
      #industry
      #former_name(s)
      #cik
      #offical company name
      #phone
      #state of incorporation
      #legal entity type
      #address
      #address2
      #city
      #state
      #state full name 
      #zip,
      #Officer names with roles

#
# Edgar is a Python Package to query the SEC Edgar Database
# https://pypi.org/project/edgar/
# 
# BeautifulSoup is a Python Package for parsing HTML and XML
# https://beautiful-soup-4.readthedocs.io/en/latest/#

In [ ]:
import lxml
from lxml import etree
import requests
from io import StringIO, BytesIO

In [ ]:
from bs4 import BeautifulSoup
from bs4 import BeautifulStoneSoup
import csv

In [ ]:
#code from https://pypi.org/project/edgar/#documents - edgar 5.4.3
#get companies by name and cik number
#get specific filing type/form (in our case Form D)
#create a list of lxml.html.ElementTree documents, one for each requested. 
#This is a LIST of things!

# Install and Call Edgar Package
# note: pip install only needed if running on Colabs or you don't have Edgar installed
# in your local environment already. 

!pip install Edgar

from edgar import Company

In [ ]:
# STEP 1.
# GET COMPANY NAMES AND CIKS FROM CSV FILE
# Use file generated from edgar_cik_company_lookup.ipynb or your own file.
# Expected file layout is two columns with a header row
# Eg:
#      SEC_company,CIK
#      EPIC THERAPEUTICS INC,0001131996 
#      PHYTERA INC,0000917054
#      C2 THERAPEUTICS INC,0001399463



# Read in companies from a csv file to create a list of lists. Each list will contain one company name.
# If running on Colabs, you will need to upload the file to the runtime to use it
# or pull it from a Google Drive:
# This section of code gets the data from my google drive
#
#from google.colab import drive
#drive.mount('/content/drive')
#f = open('/content/drive/MyDrive/Colab Notebooks/filename.csv', encoding="utf8")
#frs_text = f.read()
#f.close()


company_cik = []
with open("SEC_companies_cik1.csv", 'r') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)
    for row in csvreader:
        company_cik.append(row)

print(header)
print(company_cik[30:40])


In [ ]:
## STEP 2.
# GET FORM D CONTENT
# This code is from the edgar.py package. It takes in a company name and cik from the 
# company_cik list generated in Step 1 and passes it to the Edgar package for 
# querying at the SEC. The 'filing_type' can be set to any form name and the no_of_documents
# tells the script how many forms for each company/ form type it should return.

# The output is a list of lists of the forms as xml/html element trees which needs to be flattened 
# and then parsed for the data needed.

# More information on the classes and methods for Edgar can be found at
# https://pypi.org/project/edgar/
#
# The github repository for the code in the package is at 
# https://github.com/joeyism/py-edgar

# Parameters you can change:
#  filing_type
#  no_of_documents

docs = []
for company,cik in company_cik:
  print(company) #in case there is timeout error and the process quits we can know where we left off.
  company = Company(company, cik)
  tree = company.get_all_filings(filing_type = "D")
  forms = Company.get_documents(tree, no_of_documents=20)
  docs.append(forms)



In [ ]:
# For testing: take a look at the list
#print(docs)

In [ ]:
#Optional. You can choose a different parser to use when creating the Soup
# create the parsers for working with the ElementTree documents
#parser = etree.HTMLParser(remove_blank_text=True)
#parserxml = etree.XMLParser(remove_blank_text=True, recover=True)

In [ ]:
#STEP 3.
#CREATE DOCUMENT LIST
# unpack each item on the list of elementTrees into a string of code that you can read.
# This will create a list of the full code of the documents, one document/form = one item in the list
# These are still 'bytes' type items on the list even with the 'tostring' method. Don't ask me why.
# This is the format needed for BeautifulSoup to do its magic.

doclist = []
for item in [item for sublist in docs for item in sublist]:
    page = etree.tostring(item, pretty_print=True, encoding="UTF-8")
    doclist.append(page)



In [ ]:
#OPTIONAL - check to make sure things got put in the list
#len(doclist)

In [ ]:
#OPTIONAL - take a document or two from the list to make sure it looks like we think it should.
# change the number in the brackets to see different forms. Create a range with a colon, e.g. [23:30]
# NOTE: Python starts counting (indexing) with 0 so the first item in the list would be doclist[0].
#print(doclist[26])

In [ ]:
#STEP 4.
#GET FORM D DATA INTO CSV
# Here we take each form (page) in the list of forms (doclist), parse it and find the parts
# we want, then write those parts out to a CSV file. 
#NOTE: the 'formD.csv' file is opened in APPEND mode so if you want to build one big output file,
#you can keep running this with different doclists from your input csv files. 

with open('formD.csv', 'a') as csvfile:    
    writer = csv.writer(csvfile)

    for page in doclist:
        result = []
        xmlSoup = BeautifulStoneSoup(page)
        #This bit weeds out the forms that aren't Form D that accidentally came along for the ride.
        #Industry group is only in form D so we test for that. If it doesn't show, then
        # the exception creates a list and writes a custom error message along with the page
        # contents. 
        try:
          industry = xmlSoup.industrygrouptype.string
        except:
          errornf = "unknown form type"
          errors.append(errornf)
          errors.append(str(xmlSoup))
        #This bit actually reads through the page content and pulls out the bits we want 
        # based upon their xml tags.
        else:
          allNames = xmlSoup.findAll('relatedpersoninfo')
          companyinfo_tag = xmlSoup.primaryissuer #grab the whole primary issuer
          address_tag = xmlSoup.issueraddress #grab the address block
          signed_date = xmlSoup.signaturedate.string
          prev_names = xmlSoup.edgarpreviousnamelist #get all the previous names
          
          #industry = xmlSoup.industryGroupType.string

          #get filed on date from the SEC Header block. This is not a part of the XML so it requires extra processing.
          #This block is all text with line breaks, spaces and tabs. We have to break it up and look for the 
          #text line that contains the filed on date. If we don't find one, we create an error message to insert into the field.
          accept_tag = xmlSoup.find("acceptance-datetime")
          if accept_tag is not None:
              accept_list = accept_tag.text.split("\n")
              match = "".join([str(s) for s in accept_list if "FILED" in s])
          else:
              match = "No filing date found"
              signed_date = "No signed date found"
          result.append(match)
          result.append(signed_date)
          result.append(industry) #we append industry here to make it show up some place usable in the csv

          # Checking for previous names. If there are none, create a custom error message to insert
          # into the csv.
          try:
            for child in prev_names.children:
                child = (child.string)
                if child != "\n":
                    result.append(child)
          except: 
            errorpn = "no previous names listed"
            result.append(errorpn)

          #this bit looks through the primaryissuer block and grabs the items with no children and separates them into a field for 
          #the csv. It ignores new line characters which it will treat as their own string and you 
          #will end up with lots of empty fields (like hundreds, it filled up the sheet on the first go around).
          for child in companyinfo_tag.children:
              child = (child.string)
              if child != "\n":
                  result.append(child)

         #this bit looks through the address block and grabs the items with no children and separates them into a field for 
          #the csv. It ignores new line characters, which it will treat as their own string,  so you 
          #don't end up with lots of empty fields (like hundreds, it filled up the sheet on the first go around).
          for child in address_tag.children:
              child = (child.string)
              if child != "\n":
                  result.append(child)
          # This bit unpacks the data from the relatedpersoninfo block and creates a single name from 
          # the three fields. If middlename is empty, it enters a blank in the name.
          # Then it looks through the relationships and creates a list of all relationships. Some
          # people have more than one. The name and relationships gets entered into the csv as 
          # a list. This keeps the names and relationships together.
          for name in allNames:
              print(name)
              person = []
              relation = []
              fn = (name.find('firstname').string)
              if xmlSoup.middlename not in name:
                  mn = ""
              else:
                  mn = (name.find('middlename').string)
                  print(mn)
              ln = (name.find('lastname').string)
              rel_tag = name.relatedpersonrelationshiplist
              for child in rel_tag.children:
                  child = child.string
                  if child != "\n":
                      relation.append(child.string)
              person.append(ln+', '+fn+', '+mn+', '+', '.join(relation)) 
              result.append(person)

          
          # Here is where we write all the data requested for the current form to the csv.
          writer.writerow(result)
        
        
#always close your file!      
csvfile.close()

    

In [ ]:
#STEP 5. OPTIONAL
# CREATE ERRORS FILE
# Write out the errors to a csv file for further inspection. There may be an actual Form D or two in here
# due to malformed xml or missing data. I didn't come across any in the spot checking I did, but
# better safe than sorry.

#field_names2 = ['Error','DocCode']
#create a CSV file for the ouput
with open('errors.csv', 'a') as csvfile:
      writer = csv.writer(csvfile)
      #writer.writerow(field_names2)
      for item in errors:
        writer.writerow(item)

              
        
#always close your file!        
csvfile.close()